In [117]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from time import process_time

np.random.seed(42)

In [118]:
data = pd.read_csv("banking.csv")
dict_month = {'jan' : 1, 'feb' : 2, 'mar' : 3, 'apr' : 4, 'may' : 5, 'jun' : 6,'jul' : 7, 'aug' : 8, 'sep' : 9, 'oct' : 10, 'nov' : 11, 'dec' : 12}

data['month'] = data['month'].map(dict_month)
# convert field of dayOfweek
dict_day = {'sun' : 1, 'mon' : 2, 'tue' : 3, 'wed' : 4, 'thu' : 5, 'fri' : 6,
'sat' : 7}
data['day_of_week'] = data['day_of_week'].map(dict_day)
# conver binary fields
#default :
data.default.replace({'no' : 0, 'yes' : 1}, inplace = True)
#housing :
data.housing.replace({'no' : 0, 'yes' : 1}, inplace = True)
#loan :
data.loan.replace({'no' : 0, 'yes' : 1}, inplace = True)
# convert categories field by one host coding
marital_dummies = pd.get_dummies(data['marital'], prefix = 'marital')
marital_dummies.drop('marital_divorced', axis=1, inplace=True)
data = pd.concat([data, marital_dummies], axis=1)
job_dummies = pd.get_dummies(data['job'], prefix = 'job')
job_dummies.drop('job_unknown', axis=1, inplace=True)
data= pd.concat([data, job_dummies], axis=1)
education_dummies = pd.get_dummies(data['education'], prefix = 'education')
education_dummies.drop('education_unknown', axis=1, inplace=True)
data = pd.concat([data, education_dummies], axis=1)
contact_dummies = pd.get_dummies(data['contact'], prefix = 'contact')
#contact_dummies.drop('contact_unknown', axis=1, inplace=True)
data = pd.concat([data, contact_dummies], axis=1)
poutcome_dummies = pd.get_dummies(data['poutcome'], prefix = 'poutcome')
#poutcome_dummies.drop('poutcome_unknown', axis=1, inplace=True)
data = pd.concat([data, poutcome_dummies], axis=1)
data['pdays'] = data['pdays'].apply(lambda row: 0 if row == -1 else 1)
data.drop(['job', 'education', 'marital', 'contact', 'poutcome'], axis=1,inplace=True)

In [119]:
index = data[ (data['housing'] == 'unknown') & (data['loan'] == 'unknown') ].index
data.drop(index, inplace=True)
# data.housing.replace({'unknown': 0}, inplace=True)
# data.loan.replace({'unknown': 0}, inplace=True)

In [120]:
data.default.replace({'unknown': 0}, inplace=True)

In [121]:
p = data.groupby('default').size()
# p['unknown'] / (p[0] + p[1] + p['unknown'])
p

default
0    40195
1        3
dtype: int64

#### Sau khi có dữ liệu thuần số, hãy chia dữ liệu thành phần Training và phần Test theo tỷ lệ 8:2, trong đó với phần Test, trường y sẽ không dùng đến.

In [122]:
target = 'y'
X = data.drop(target, axis=1)
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)

## A

#### Sử dụng mô hình hồi quy logistic với phần dữ liệu Training

In [128]:
cls = LogisticRegression()
start1 = process_time()
cls.fit(X_train, y_train)
end1 = process_time()
print(end1 - start1)

0.09375


d:\APP\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Dự đoán và tính metrics (Logistic)

In [124]:
y_predict = cls.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      7139
           1       0.65      0.35      0.46       901

    accuracy                           0.91      8040
   macro avg       0.79      0.66      0.70      8040
weighted avg       0.89      0.91      0.89      8040



In [125]:
cm = np.array(confusion_matrix(y_test, y_predict, labels=[0, 1]))
confusion = pd.DataFrame(cm, index=["Khong dang ky", "co dang ky"], columns=["khong dang ky", "co dang ky"])
confusion

,khong dang ky,co dang ky
Khong dang ky,6967,172
co dang ky,584,317


## B

### Sử dụng mô hình Naïve Bayes phù hợp với tập huấn luyện như trên, sau đó chạy dự đoán với tập Test và tính độ chính xác của mô hình, như ý a.

In [129]:
cls = GaussianNB()
start2 = process_time()
cls.fit(X_train, y_train)
end2 = process_time()
print(end2 - start2)

0.015625


#### Dự đoán và tính metrics (Naive Bayes)

In [127]:
y_predict = cls.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.94      0.86      0.90      7139
           1       0.33      0.57      0.42       901

    accuracy                           0.82      8040
   macro avg       0.64      0.71      0.66      8040
weighted avg       0.87      0.82      0.84      8040



## C

#### So sánh thời gian chạy cũng như độ chính xác của hai phương pháp.

Thời gian chạy phương pháp:
- Logistic regression: 0.09375 (s)
- Naive Bayes: 0.015625 (s)

Độ chính xác:
- Logistic regression: 0.91
- Naive Bayes: 0.82

=> 
- Logistic regression có độ chính xác cao hơn.
- Naive bayes nhanh hơn.
- Precision và recall của cả 2 phương pháp đối với class `1` đều thấp, đối với class `0` thì cao, do chưa có bước xử lý dữ liệu không cân bằng.